In [1]:
import os, sys

In [2]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


In [3]:
import argparse
import yaml
import numpy as np
from pprint import pprint

In [4]:
import argparse
import os

import torch
import yaml
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
# from tqdm import tqdm

In [5]:
from utils.model import get_model, get_vocoder, get_param_num
from utils.tools import to_device, log, synth_one_sample
from model import FastSpeech2Loss
from dataset import Dataset
from utils.auto_tqdm import tqdm

from evaluate import evaluate

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument("--restore_step", type=int, default=0)
parser.add_argument(
    "-p",
    "--preprocess_config",
    type=str,
    required=True,
    help="path to preprocess.yaml",
)
parser.add_argument(
    "-m", "--model_config", type=str, required=True, help="path to model.yaml"
)
parser.add_argument(
    "-t", "--train_config", type=str, required=True, help="path to train.yaml"
)

argString = '-p ./config/Ego4D_final_v6/0625a_preprocess.yaml -m ./config/Ego4D_final_v6/0625a_model.yaml -t ./config/Ego4D_final_v6/0625a_train.yaml'
# args = parser.parse_args()
args = parser.parse_args(argString.split())

In [8]:
args

Namespace(restore_step=0, preprocess_config='./config/Ego4D_final_v6/0625a_preprocess.yaml', model_config='./config/Ego4D_final_v6/0625a_model.yaml', train_config='./config/Ego4D_final_v6/0625a_train.yaml')

In [ ]:
# Read Config
preprocess_config = yaml.load(
    open(args.preprocess_config, "r"), Loader=yaml.FullLoader
)
model_config = yaml.load(open(args.model_config, "r"), Loader=yaml.FullLoader)
train_config = yaml.load(open(args.train_config, "r"), Loader=yaml.FullLoader)
configs = (preprocess_config, model_config, train_config)

In [10]:
# main()

In [11]:
print("Prepare training ...")

preprocess_config, model_config, train_config = configs

# Get dataset
dataset = Dataset(
    "train.txt", 'train', preprocess_config, train_config, sort=True, drop_last=True
)

Prepare training ...


In [12]:
len(dataset)

27292

In [13]:
pprint(dataset[0])

{'duration': array([12,  5,  4,  6,  2,  3,  2,  3,  7,  3,  5,  4,  3,  2,  5,  3,  2,
        7, 19], dtype=int64),
 'energy': array([-0.28664967, -0.81794465, -0.21089569,  1.7952452 ,  2.4001818 ,
        1.5255114 ,  0.955319  ,  0.7556556 ,  1.7037253 ,  0.32923424,
       -0.7877115 , -0.7344083 , -0.21618795, -0.43786454, -0.3020548 ,
       -0.24257718, -0.37000364, -0.7311336 , -0.43382198], dtype=float32),
 'id': '634b2f0d-2cca-4bc6-bb13-905c0c2748e7',
 'mel': array([[ -6.924207 ,  -7.505341 ,  -7.517166 , ..., -11.512925 ,
        -11.512925 , -11.512925 ],
       [ -6.4425254,  -7.5613546,  -7.367668 , ..., -11.512925 ,
        -11.512925 , -11.512925 ],
       [ -6.4373703,  -7.5682425,  -7.524733 , ..., -11.512925 ,
        -11.512925 , -11.512925 ],
       ...,
       [ -5.631855 ,  -4.829329 ,  -4.142794 , ..., -11.512925 ,
        -11.512925 , -11.512925 ],
       [ -6.1263156,  -6.160098 ,  -5.7407236, ..., -11.512925 ,
        -11.512925 , -11.512925 ],
       [ -6.

In [14]:
dataset[0]['speaker_embedding'].shape

(192,)

In [15]:
dataset[0]['pitch'].shape

(19,)

In [16]:
dataset[0]['mel'].shape

(97, 80)

In [17]:
# batch_size = train_config["optimizer"]["batch_size"]
batch_size = 6
group_size = 4  # Set this larger than 1 to enable sorting in Dataset
assert batch_size * group_size < len(dataset)
loader = DataLoader(
    dataset,
    batch_size=batch_size * group_size,
    shuffle=True,
    collate_fn=dataset.collate_fn,
)

In [18]:
batch_size

6

In [19]:
for data in loader:
    break

1223123123
speaker_embeddings.dtype: float32


In [20]:
len(data)

1

In [21]:
# return (
#     ids,
#     raw_texts,
#     speakers,
#     texts,
#     text_lens,
#     max(text_lens),
#     mels,
#     mel_lens,
#     max(mel_lens),
#     pitches,
#     energies,
#     durations,
#     speaker_embeddings,
# )
# [12] 对应speaker embedding
for item in data:
    print(item[12].shape)

(24, 192)


In [22]:
for item in data:
    print(item[11].shape)

(24, 80)


In [23]:
for item in data:
    print(item[3].shape)

(24, 80)


In [24]:
for item in data:
    print(item[4].shape)

(24,)


In [25]:
# Prepare model
model, optimizer = get_model(args, configs, device, train=True)
model = nn.DataParallel(model)
num_param = get_param_num(model)
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)
print("Number of FastSpeech2 Parameters:", num_param)

# Load vocoder
vocoder = get_vocoder(model_config, device)

Number of FastSpeech2 Parameters: 35208513
Removing weight norm...


In [26]:
# # Init logger
# for p in train_config["path"].values():
#     os.makedirs(p, exist_ok=True)
# train_log_path = os.path.join(train_config["path"]["log_path"], "train")
# val_log_path = os.path.join(train_config["path"]["log_path"], "val")
# os.makedirs(train_log_path, exist_ok=True)
# os.makedirs(val_log_path, exist_ok=True)
# train_logger = SummaryWriter(train_log_path)
# val_logger = SummaryWriter(val_log_path)


In [27]:
# Training
step = args.restore_step + 1
epoch = 1
grad_acc_step = train_config["optimizer"]["grad_acc_step"]
grad_clip_thresh = train_config["optimizer"]["grad_clip_thresh"]
total_step = train_config["step"]["total_step"]
log_step = train_config["step"]["log_step"]
save_step = train_config["step"]["save_step"]
synth_step = train_config["step"]["synth_step"]
val_step = train_config["step"]["val_step"]

In [31]:
1

1

In [28]:
for batchs in loader:
    for batch in batchs:
        batch = to_device(batch, device)
        temp_batch = (batch[2:])
        output = model(*(batch[2:]))
        break
    break

1223123123
speaker_embeddings.dtype: float32
=====
output.shape:  torch.Size([24, 85, 256])
speaker_embeddings.shape: torch.Size([24, 192])
max_src_len: 85
=====
speaker_embeddings.dtype:  torch.float32
expanded_embeddings.shape:  torch.Size([24, 85, 256])


In [34]:
len(output)

10

In [35]:
output

(tensor([[[ 8.4385e-01, -2.9523e-01, -2.3488e-02,  ..., -3.5501e-01,
           -7.0581e-01,  6.6764e-01],
          [ 4.6136e-01, -8.2919e-01, -2.8015e-01,  ...,  9.5114e-02,
           -7.4012e-01,  1.0405e+00],
          [ 5.0622e-01, -4.0070e-01, -6.2528e-01,  ...,  2.2488e-01,
           -7.9387e-01,  8.3428e-01],
          ...,
          [ 9.8646e-01, -6.5670e-01,  8.0198e-02,  ...,  1.5776e-01,
           -3.2110e-02,  6.7201e-01],
          [ 7.3551e-01, -4.0821e-01,  1.2232e-02,  ...,  2.0419e-01,
           -3.2390e-01,  7.3182e-01],
          [ 7.0838e-01, -5.6651e-02,  1.7995e-01,  ..., -1.2784e-01,
           -2.8903e-01,  9.0250e-01]],
 
         [[-1.0785e-01, -4.9586e-01, -7.3096e-01,  ...,  3.9264e-01,
           -2.5788e-01,  7.8091e-01],
          [ 2.5060e-01, -1.1282e+00, -2.4798e-01,  ...,  3.5738e-01,
           -1.1274e-02,  9.2995e-01],
          [ 1.8820e-01, -9.4311e-01,  8.4477e-02,  ...,  4.6814e-01,
           -3.3119e-01,  9.6642e-01],
          ...,
    

In [29]:
len(batch)

13

In [30]:
batch[-2].shape

torch.Size([24, 85])

In [ ]:
batch[-1].shape

In [ ]:
len(temp_batch)

In [ ]:
temp_batch

In [ ]:
temp_batch[-1].shape

In [ ]:
temp_batch[3]

In [ ]:
temp_batch[2]